___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg" width="350px" height="180px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2021
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://miro.medium.com/max/1400/1*QJZ6W-Pck_W7RlIDwUIN9Q.jpeg</p>

### <font color= #2E9AFE> Tema: XGBoost - Regresión y Clasificación</font>


#### Características
- Consiste en un ensamblado **secuencial** de árboles de decisión. Los árboles se agregan secuencialmente con la finalidad de aprender del resultado de árboles anteriores y corregir el error producido por los mismos, hasta que ya no se pueda corregir dicho error.
- Utiliza el procesamiento en paralelo, poda de árboles, manejo de valores nulos, regularización para evitar el sobreajuste del modelo 

#### Aplicaciones
Uno de los sectores en los que se aplican este tipo de algoritmos es el financiero.
Algunos ejemplos de su aplicación son: segmentación de clientes, detección de fraudes, pronóstico de ventas, autenticación de clientes y análisis de comportamiento de mercados. 

Un área de interés es identificar clientes a quienes otorgar una tarjeta de crédito, esto es crítico para los bancos, ya que una selección incorrecta de estos clientes podría derivar en un incremento de su cartera vencida.


#### Parámetros a optimizar

- Tasa de aprendizaje (eta)
- Profundidad del árbol
- Gamma (para podar el árbol)
- Lambda (regularización)
- Min child weight
- scale_pos_weigth

#### Ventajas de XGBoost

- Puede manejar grandes bases de datos con muchas variables
- Puede manejar datos nulos
- Resultados muy precisos
- Rápidos de ejecutar

#### Desventajas de XGBoost
- Puede consumir muchos recursos computacionales, por lo que se recomienda determinar antes cuáles son las variables que aportarán más información al modelo
- Se deben ajustar correctamente los parámetros del algoritmo
- Es conveniente transformar los datos a que todos sean numéricos

## Clasificación

### Datos

Vamos a utilizar los datos de **IBM Base Samples** para predecir si un cliente va a dejar de usar los servicios de IBM. 

Los datos se pueden descargar de:
https://www.kaggle.com/yeanzc/telco-customer-churn-ibm-dataset

Se tienen 7043 observaciones y 33 variables

- CustomerID: ID único que identifica a cada cliente.
- Count: valor utilizado en informes para sumar el número de clientes en un conjunto determinado.
- Country: donde vive el cliente
- State: estado donde vive el cliente
- Ciudad: ciudad donde vive el cliente
- Zip Code: zip code donde vive el cliente
- Lat Long: La latitud y longitud combinadas de la residencia principal del cliente.
- Latitude: latitud 
- Longitude: longitud
- Gender: género
- Senior Citizen: Indica si el cliente es 65 o mayor: Yes, No
- Partner: Indica si el cliente tiene un socio: Yes, No
- Dependents: Indica si el cliente tiene algún dependiente: yes, no. (Hijos, padres, abuelos, etc). 
- Tenure Months: Indicates the total amount of months that the customer has been with the company by the end of the quarter specified above.

Phone Service: Indicates if the customer subscribes to home phone service with the company: Yes, No

Multiple Lines: Indicates if the customer subscribes to multiple telephone lines with the company: Yes, No

Internet Service: Indicates if the customer subscribes to Internet service with the company: No, DSL, Fiber Optic, Cable.

Online Security: Indicates if the customer subscribes to an additional online security service provided by the company: Yes, No

Online Backup: Indicates if the customer subscribes to an additional online backup service provided by the company: Yes, No

Device Protection: Indicates if the customer subscribes to an additional device protection plan for their Internet equipment provided by the company: Yes, No

Tech Support: Indicates if the customer subscribes to an additional technical support plan from the company with reduced wait times: Yes, No

Streaming TV: Indicates if the customer uses their Internet service to stream television programing from a third party provider: Yes, No. The company does not charge an additional fee for this service.

Streaming Movies: Indicates if the customer uses their Internet service to stream movies from a third party provider: Yes, No. The company does not charge an additional fee for this service.

Contract: Indicates the customer’s current contract type: Month-to-Month, One Year, Two Year.

Paperless Billing: Indicates if the customer has chosen paperless billing: Yes, No

Payment Method: Indicates how the customer pays their bill: Bank Withdrawal, Credit Card, Mailed Check

Monthly Charge: Indicates the customer’s current total monthly charge for all their services from the company.

Total Charges: Indicates the customer’s total charges, calculated to the end of the quarter specified above.

Churn Label: Yes = the customer left the company this quarter. No = the customer remained with the company. Directly related to Churn Value.

Churn Value: 1 = the customer left the company this quarter. 0 = the customer remained with the company. Directly related to Churn Label.

Churn Score: A value from 0-100 that is calculated using the predictive tool IBM SPSS Modeler. The model incorporates multiple factors known to cause churn. The higher the score, the more likely the customer will churn.

CLTV: Customer Lifetime Value. A predicted CLTV is calculated using corporate formulas and existing data. The higher the value, the more valuable the customer. High value customers should be monitored for churn.

Churn Reason: A customer’s specific reason for leaving the company. Directly related to Churn Category.

In [ ]:
#Librerías
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import train_test_split, GridSearchCV

In [ ]:
#Importar datos
data = pd.read_excel('C:\\Users\\rodrigsa\\OneDrive - HP Inc\\ITESO\\TallerModelado\\Autum2021\\Telco_customer_churn.xlsx')

In [ ]:
#vistazo de los datos
data.head()

In [ ]:
data.dtypes

**Limpiar y procesamiento de datos**

In [ ]:
#quitar espacio en blanco de los nombres de las columnas y reemplazar por guión bajo
data.columns = data.columns.str.replace(' ', '_')


In [ ]:
#La columna "Total Charges" parece ser una variable categórica (object) pero al hacer un vistazo 
#en los datos (data.head()) parece haber datos numéricos
data['Total_Charges'].unique()

In [ ]:
#convertimos la variable Total Charges a numérica
data['Total_Charges'] = pd.to_numeric(data['Total_Charges'])

In [ ]:
#Parece haber filas con espacio en blanco y por eso toma a la variable como categórica
# Sustituimos esos valores con cero
data.loc[(data['Total_Charges']==' '),'Total_Charges']=0
#convertimos a tipo de dato numérico
data['Total_Charges'] = pd.to_numeric(data['Total_Charges'])
data.dtypes

In [ ]:
#Existen 4 variables que debemos remover
data.drop(['Churn_Label','Churn_Reason','Churn_Score','CLTV'], axis=1, inplace=True)

In [ ]:
#Eliminar variables que tienen muy poca varianza (que contienen muy pocos valores únicos)
data.nunique()

In [ ]:
#quitamos CustomerID, Lat Long, count, country y state
data.drop(['CustomerID','Lat_Long','Count','Country','State'], axis=1, inplace=True)

In [ ]:
data.nunique()

In [ ]:
#quitar espacio en blanco de los valores de las columnas y reemplazar por guión bajo
data.replace(' ','_', regex=True, inplace=True)

In [ ]:
#Separar nuestras X de nuestras Y
X = data.drop('Churn_Value', axis=1).copy()
X.head(3)

In [ ]:
y = data['Churn_Value'].copy()

In [ ]:
#Crear variables dummy. Convertir una varialbe categórica en múltiples columnas con valores binarios
X_nuevo = pd.get_dummies(X, columns=['City','Gender','Senior_Citizen','Partner','Dependents',
                           'Phone_Service','Multiple_Lines','Internet_Service','Online_Security',
                          'Online_Backup','Device_Protection','Tech_Support','Streaming_TV',
                          'Streaming_Movies','Contract','Paperless_Billing','Payment_Method'])
X_nuevo.head(3)

In [ ]:
#Viendo si nuestra variable a predecir tiene datos imbalanceados
sum(y)/len(y)

**Construcción del modelo**

In [ ]:
#Dividir los datos en train y test
X_train, X_test, y_train, y_test = train_test_split(X_nuevo, y, random_state=42, stratify=y)

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
sum(y_test)/len(y_test)

In [ ]:
#Construir el xgboost
#inicializar objeto de clasificación
modelo = xgb.XGBClassifier(objective='binary:logistic', missing=1, seed=42)
#Entrenar modelo
modelo.fit(X_train, y_train, verbose=True,
          early_stopping_rounds=10, #parar de construir más árboles si no mejora la situación de los residuales
          eval_metric='aucpr',
          eval_set=[(X_test, y_test)])

In [ ]:
#ver cómo funciona el modelo entrenado con los datos de prueba
plot_confusion_matrix(modelo, X_test, y_test, values_format='d', display_labels=["No se fue","Se fue"])


En la matriz de confusión vemos que de 1294 personas que no se fueron, 1148 (88%) fueron clasificados correctamente. 

Y de los 467 personas que sí se fueron, 255 (54%) fueron clasificados correctamente. 

Entonces el modelo de XGBoost no fue tan bueno. Parte del problema es que los datos están imbalanceados. 

Ya que el hecho de que se vaya la gente y deje los servicios de IBM cuesta dinero, queremos capturar más la información de las personas que se fueron. La buena noticia es que XGBoost tiene un parámetro, 
*scale_pos_weight*, que ayuda con datos imbalanceados. 

Utilicemos **cross-validation** para optimizar los parámetros. 

#### Cross validation y GridSearch para optimizar los hiper-parámetros

XGBoost tiene muchísimos hiper-parámetros a ajustar ya sean:
- max_depth 
- learning_rate (eta)
- gamma
- reg_lambda

In [ ]:
#como es bien tardado el proceso del ajuste de hiperparámetros, en lugar de optimizar todo
#de una vez, optimicé parámetros por secciones para hacerlo más rápido

#Ronda 1
#param_grid={
#    'max_depth':[3,4,5],
#    'learning_rate':[0.1,0.01,0.05],
#    'gamma':[0, 0.25,1],
#    'reg_lambda':[0,1,10],
#    'scale_pos_weight':[1,3,5]
#}

#Ronda 2
param_grid={
    'max_depth':[4],
    'learning_rate':[0.1,0.5,1],
    'gamma':[0.25],
    'reg_lambda':[10,20,100],
    'scale_pos_weight':[3]
}

optimal_params = GridSearchCV(
                            estimator=xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            subsample=0.9,
                            colsample_bytree=0.5),
                            param_grid=param_grid,
                            scoring='roc_auc',
                            verbose=0,
                            n_jobs=10,
                            cv=3)

optimal_params.fit(X_train, y_train,
                  early_stopping_rounds=10,
                  eval_metric='auc',
                  eval_set=[(X_test, y_test)],
                  verbose=False)
optimal_params.best_params_

{'gamma': 0.25,
 'learning_rate': 0.1,
 'max_depth': 4,
 'reg_lambda': 10,
 'scale_pos_weight': 3}

In [ ]:
#Construir y evaluar el XGBoost con los hiperparámetros óptimos
modelo = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                           gamma=0.25,
                           learn_rate=0.1,
                           max_depth=4,
                           reg_lambda=10,
                           scale_pos_weight=3,
                            subsample=0.9,
                            colsample_bytree=0.5)

modelo.fit(X_train, y_train,
                  early_stopping_rounds=10,
                  eval_metric='auc',
                  eval_set=[(X_test, y_test)],
                  verbose=False)

In [ ]:
#ver cómo funciona el modelo entrenado con los datos de prueba
plot_confusion_matrix(modelo, X_test, y_test, values_format='d', display_labels=["No se fue","Se fue"])


En la matriz de confusión vemos que de 1294 personas que no se fueron, 933 (72%) fueron clasificados correctamente. 

Y de los 467 personas que sí se fueron, 387 (82%) fueron clasificados correctamente

#### Un poco de bibliografía...
https://www.kaggle.com/prashant111/a-guide-on-xgboost-hyperparameters-tuning